# 在浏览器中执行Python代码-最大化控件

使用`%%javascript`魔法命令可以在浏览器中执行Javascript代码。例如下面的Javascript代码将当前单元的背景修改为随机的浅颜色。

In [58]:
%%javascript
/* 按Ctrl+Enter运行本单元 */
var cell = Jupyter.notebook.get_selected_cell();
var r = Math.floor(Math.random() * 56 + 200).toString(16);
var g = Math.floor(Math.random() * 56 + 200).toString(16);
var b = Math.floor(Math.random() * 56 + 200).toString(16);
console.log(r, g, b);
$(cell.element).find("div.input_area").css("background-color", "#" + r + g + b);

<IPython.core.display.Javascript object>

但是作为Python程序员，我们还是希望能使用Python编写这些代码。可以使用`flexx`提供的`py2js()`将Python代码转换成Javascript，然后通过`IPython.display.display_javascript()`执行。下面是实现该方法的魔法命令`%%py2js`。

In [41]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def py2js(line, cell):
    from flexx.pyscript import py2js
    from IPython.display import display_javascript
    js = py2js(cell)
    run_js = "(function(){%s})();" % js
    display_javascript(run_js, raw=True)

下面是使用Python实现随机修改背景色的代码。在这段代码中，

* 使用了Python列表推导式和字符串的`join()`方法。
* 使用`print()`代替`console.log()`。
* 由于`$`不是有效标识符，这里使用`jQuery`代替。

In [61]:
%%py2js
# 按Ctrl+Enter运行本单元
cell = Jupyter.notebook.get_selected_cell()
color = "#" + "".join([int(Math.random() * 56 + 200).toString(16) for _ in range(3)])
print(color)
jQuery(cell.element).find("div.input_area").css("background-color", color)

本书的`py2js`模块中已经提供了`%%py2js`魔法命令，可以直接使用`%load_ext`载入。

In [11]:
%load_ext py2js

## 最大化控件

下面使用`%%py2js`编写一个最大化`ipywidgets`控件的函数。执行下面的单元之后：

* 运行`window.full_widget.fullscreen()`最大化控件；
* 按ESC或者运行`window.full_widget.cancel()`返回。

In [ ]:
%%py2js

style_center = """
position:fixed;
top:50%;
left:50%;
transform:translate(-50%, -50%);
"""

style_full = """
height:100vh;
width:100vw;
background-color:rgba(170, 170, 170, 0.9)
;z-index:100;"""

html_full = '<div id="full_widget"><div id="center_widget"></div></div>'

class FullWidget:
    def __init__(self):
        pass
        
    def fullscreen(self):
        self.widgets = jQuery(jQuery(".widget-area:visible")[-1])
        self.parents = self.widgets.parent()
        if len(jQuery("#full_widget")) > 0:
            return
        jQuery(html_full).appendTo(jQuery('body'))
        jQuery('#full_widget').attr('style', style_full + style_center)
        jQuery('#center_widget').attr('style', style_center)
        self.widgets.appendTo(jQuery('#center_widget'))

        def on_key(event):
            if event.which == 27:
                self.cancel()

        jQuery("body").keydown(on_key)
        
    def cancel(self):
        if len(jQuery("#full_widget")) > 0:
            self.widgets.insertAfter(self.parents.find(".input"))
            jQuery('#full_widget').remove()
            jQuery("body").unbind("kewdown")
        
if not window.FullWidget:
    window.FullWidget = FullWidget
    window.full_widget = FullWidget()

下面使用`ipywidgets`创建一个简单的界面，点击`^`最大化控件。

In [9]:
from ipywidgets import Button, IntText, HBox, Layout

fullscreen_button = Button(description="^", layout=Layout(width="15px"))
count_button = Button(description="Count Up")
counter = IntText(value=0)

def count_up(event):
    counter.value += 1
    
def full_screen(event):
    from IPython.display import display_javascript, clear_output
    display_javascript("window.full_widget.fullscreen()", raw=True)
    clear_output()
    
fullscreen_button.on_click(full_screen)
count_button.on_click(count_up)
HBox([fullscreen_button, count_button, counter])

`py2js`中提供了`make_fullscreen_button()`函数创建上述最大化按钮。

In [10]:
from py2js import make_fullscreen_button
fullscreen_button = make_fullscreen_button()
fullscreen_button